In [ ]:
import pandas as pd
import geopandas
from folium import Circle, Marker, Icon, Map
from pymongo import MongoClient, GEOSPHERE
conn = MongoClient("localhost:27017")
db = conn.get_database("madrid")
import random
from folium import Circle, Marker, Icon, Map

In [ ]:
df_restaurants = pd.read_csv("data/restaurants.csv", index_col=0)
df_snacks = pd.read_csv("data/snacks.csv", index_col=0)
df_culture = pd.read_csv("data/culture.csv", index_col=0)
df_party = pd.read_csv("data/party.csv", index_col=0)
df_drinks = pd.read_csv("data/drinks.csv", index_col=0)
df_outdoors = pd.read_csv("data/outdoors.csv", index_col=0)
df_leisure = pd.read_csv("data/leisure.csv", index_col=0)

In [ ]:
#list(df_restaurants.CP.unique())

In [ ]:
leisure = db.get_collection("leisure")
outdoors = db.get_collection("outdoors")
restaurants = db.get_collection("restaurants")
drinks = db.get_collection("drinks")
party = db.get_collection("party")
snacks = db.get_collection("snacks")
culture = db.get_collection("culture")

transport = db.get_collection("transport")


## Team-building

In [ ]:
db.list_collection_names()

In [ ]:
intersection = set(list(df_leisure.CP.unique())).intersection(set(list(df_restaurants.CP.unique())))

In [ ]:
cp = list(intersection)

In [ ]:
cp_random = random.choice(cp)

In [ ]:
leis = list(leisure.find({"CP":str(int(cp_random))}))

In [ ]:
rest = list(restaurants.find({"CP":str(int(cp_random))}))

In [ ]:
l_random = random.choice(leis)
r_random = random.choice(rest)

In [ ]:
l_random

In [ ]:
r_random

## Friends

In [ ]:
intersection = set(list(df_drinks.CP.unique())).intersection(set(list(df_party.CP.unique())))

cp = list(intersection)

cp_random = random.choice(cp)

drinks_ = list(drinks.find({"CP":str(int(cp_random))}))

party_ = list(party.find({"CP":str(int(cp_random))}))

d_random = random.choice(drinks_)
p_random = random.choice(party_)

In [ ]:
d_random

In [ ]:
p_random

In [ ]:
df1 = pd.DataFrame.from_dict(d_random)
df2 = pd.DataFrame.from_dict(p_random)
df = pd.concat([df1,df2])

In [ ]:
#df = df.set_index("name")
df =df.drop(["geometry"], axis = 1)

In [ ]:
df = df.drop_duplicates()

In [ ]:
df

## General

In [ ]:
def planes(df_1, df_2, coll_1, coll_2):
    
    intersection = set(list(df_1.CP.unique())).intersection(set(list(df_2.CP.unique())))

    cp = list(intersection)

    cp_random = random.choice(cp)

    plan1 = list(coll_1.find({"CP":str(int(cp_random))}))

    plan2 = list(coll_2.find({"CP":str(int(cp_random))}))
    
    p1_random = random.choice(plan1)
    p2_random = random.choice(plan2)

    df1 = pd.DataFrame.from_dict(p1_random)
    df2 = pd.DataFrame.from_dict(p2_random)
    df = pd.concat([df1,df2])
    
    df = df.set_index("_id")
    df =df.drop(["geometry"], axis = 1)
    df = df.drop_duplicates()
    
    return df

In [ ]:
def get_df(df):
    df = df.set_index("name")
    return df.drop(["place", "CP", "latitude", "longitude"], axis=1)

In [ ]:
def get_map(df):
    map_1 = Map(location=[list(df.latitude.unique())[0],list(df.longitude.unique())[0]],zoom_start=15)
    for i, row in df.iterrows():
        geom = {
                    "location":[row["latitude"], row["longitude"]],
                    "tooltip" : row["name"]
                }
        icon = Icon(color = "white",
                                prefix = "fa",
                                icon = "trophy",
                                icon_color = "black"
                )


        Marker(**geom,icon = icon ).add_to(map_1)
        
    return map_1

In [ ]:
def type_of_plan(plan):
    
    options_team_building = { "option_1" : [df_outdoors, df_restaurants, outdoors, restaurants],
                             "option_2" : [df_leisure, df_restaurants, leisure, restaurants]
                            }

    options_friends = { "option_1" : [df_restaurants, df_drinks, restaurants, drinks],
                        "option_2" : [df_outdoors, df_restaurants, outdoors, restaurants],
                        "option_3" : [df_leisure, df_restaurants, leisure, restaurants],
                        "option_4" : [df_party, df_restaurants, party, restaurants],
                        "option_5" : [df_drinks, df_party, drinks, party],
                        "option_6" : [df_snacks, df_leisure, snacks, leisure],
                        "option_7" : [df_outdoors, df_snacks, outdoors, snacks]
                     }

    options_couple = { "option_1" : [df_restaurants, df_drinks, restaurants, drinks],
                       "option_2" : [df_leisure, df_restaurants, leisure, restaurants],
                       "option_3" : [df_culture, df_restaurants, culture, restaurants],
                       "option_4" : [df_culture, df_snacks, culture, snacks]
                     }

    options_individual = { "option_1" : [df_culture, df_restaurants, culture, restaurants],
                           "option_2" : [df_culture, df_snacks, culture, snacks]
                         }

    options_family = {  "option_1" : [df_culture, df_restaurants, culture, restaurants],
                        "option_2" : [df_culture, df_snacks, culture, snacks],
                        "option_3" : [df_outdoors, df_restaurants, outdoors, restaurants],
                        "option_4" : [df_outdoors, df_snacks, outdoors, snacks],
                        "option_5" : [df_leisure, df_restaurants, leisure, restaurants],
                        "option_6" : [df_leisure, df_snacks, leisure, snacks]
                     }
    
    if plan == "family":
        options =  options_family
    elif plan == "friends":
        options = options_friend
    elif plan == "individual":
        options = options_individual
    elif plan == "couple":
        options = options_couple
    elif plan == "team building":
        options = options_team_building
     
    opt = random.choice(list(options.keys()))
    args = options[opt]
    arg_1 = args[0]  
    arg_2 = args[1]  
    arg_3 = args[2]  
    arg_4 = args[3]  
    
    return planes(arg_1, arg_2, arg_3, arg_4)

In [ ]:
plan = type_of_plan("couple")

In [ ]:
get_df(plan)

In [ ]:
get_map(plan)